In [1]:
%load_ext autoreload
%autoreload 2

import os
import napari

import sys
sys.path.append('..')

from tracks_interactions.widget.widget_main import TrackGardener

In [2]:
os.environ['NAPARI_PERFMON'] = '0'

from napari.settings import get_settings
settings = get_settings()
settings.experimental.async_
#settings.experimental.async_=True

False

In [3]:
viewer = napari.Viewer()

track_gardener = TrackGardener(viewer)

viewer.window.add_dock_widget(track_gardener,area='right',name='Track Gardener')